In [ ]:
!pip install underthesea

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 7.6 MB 4.1 MB/s 
     |████████████████████████████████| 581 kB 68.1 MB/s 
     |████████████████████████████████| 235 kB 74.2 MB/s 
     |████████████████████████████████| 965 kB 66.6 MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import io
import random
import string 
import warnings
import numpy as np
import pandas as pd
import underthesea
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import warnings
from nltk.translate.bleu_score import sentence_bleu
warnings.filterwarnings('ignore')

In [ ]:
import nltk
nltk.download('popular', quiet=True)

True

In [ ]:
dataset_chatbot_1 = pd.read_csv("/content/drive/MyDrive/chatbot data.csv")
dataset_chatbot_2 = pd.read_csv("/content/drive/MyDrive/dữ liệu chatbot question-answer short style.csv")

In [ ]:
dataset_chatbot_1.isin(['?']).sum(axis=0)

Unnamed: 0    0
user_a        0
user_b        0
dtype: int64

In [ ]:
dataset_chatbot_2.isin(['?']).sum(axis=0)

Unnamed: 0    0
user_a        2
user_b        2
dtype: int64

In [ ]:
dataset_chatbot_2['user_a'] = dataset_chatbot_2['user_a'].replace('?',np.nan)
dataset_chatbot_2['user_b'] = dataset_chatbot_2['user_b'].replace('?',np.nan)

In [ ]:
dataset_chatbot_1.dropna(inplace=True)
dataset_chatbot_2.dropna(inplace=True)

In [ ]:
dataset_chatbot_1.head()

,Unnamed: 0,user_a,user_b
0,0,bạn tên gì?,Mình tên Trường <eos> còn bạn tên gì?
1,1,Tui tên Nam <eos> Bạn học ở đâu?,tui học ở TDTU nè <eos> còn bạn học ở trường n...
2,2,tui học ở TDTU luôn <eos> Bạn học ngành gì,mình học Công Nghệ Thông Tin
3,3,Chuyên ngành gì vậy bạn ?,mình học bên Kỹ Thuật Phần Mềm á <eos> còn bạn...
4,4,mình học Quản Trị Kinh Doanh <eos> Bạn là ngườ...,Mình là người Đồng tháp <eos> bạn biết Đồng Th...


In [ ]:
dataset_chatbot_2.head()

,Unnamed: 0,user_a,user_b
0,0,Thích mẫu người nào,"Dễ thương, tóc dài, da trắng"
1,1,Có crush ai không,Có 1 bạn cùng lớp
2,2,Tại sao lại thích bạn dó,Vì đáp ứng những yêu cầu của tao
3,3,Có hay nói chuyện không,Hay nhắn tin messenger
4,4,Bạn kia có bắt chuyện trước không,Có đôi khi


In [ ]:
question_tokens = []

for data in dataset_chatbot_1['user_a']:
    if "<eos>" in data.split():
        data = data.replace(" <eos> ",", ")
    question_tokens.append(data)

for data in dataset_chatbot_2['user_a']:
    question_tokens.append(data)

print(question_tokens[:5])
print(len(question_tokens))

['bạn tên gì?', 'Tui tên Nam, Bạn học ở đâu?', 'tui học ở TDTU luôn, Bạn học ngành gì', 'Chuyên ngành gì vậy bạn ?', 'mình học Quản Trị Kinh Doanh, Bạn là người ở đâu vậy ?']
7274


In [ ]:
answer_tokens = []
for data in dataset_chatbot_1['user_b']:
    if "<eos>" in data.split():
        data = data.replace(" <eos> ",", ")
    answer_tokens.append(data)

for data in dataset_chatbot_2['user_b']:
    answer_tokens.append(data)

print(answer_tokens[:5])
print(len(answer_tokens))

['Mình tên Trường, còn bạn tên gì? ', 'tui học ở TDTU nè, còn bạn học ở trường nào ?', 'mình học Công Nghệ Thông Tin', 'mình học bên Kỹ Thuật Phần Mềm á, còn bạn học ngành gì ?', 'Mình là người Đồng tháp, bạn biết Đồng Tháp không ?']
7274


In [ ]:
def removePunctuations(sen):
    temp_l = sen.split()
    i = 0
    j = 0
    for word in temp_l :
        j = 0
        for l in word :
            if l in string.punctuation:
                word = word.replace(l," ")
            j += 1
        temp_l[i] = word.lower()
        i=i+1   
    content = " ".join(temp_l)
    return content

In [ ]:
def get_onegram_token(tokens):
    return [token for token in tokens]

def one_gram(text):
    return get_onegram_token(underthesea.word_tokenize(removePunctuations(text.lower())))

In [ ]:
def response(user_response):
    bot_response=''
    question_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=one_gram)
    tfidf = TfidfVec.fit_transform(question_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    question_tokens.pop()
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        bot_response=bot_response + "Tôi xin lỗi! Tôi không hiểu bạn nói gì?"
        return bot_response
    else:
        bot_response = bot_response + answer_tokens[idx]
        return bot_response

In [ ]:
bleu_score_list = []
for index ,question in enumerate(question_tokens[:500]):
    res = response(question)
    bleu_score_list.append(sentence_bleu(answer_tokens[index], res))

In [ ]:
bleu_score_list[0:10]

[0.7598356856515925,
 0.8545740127924681,
 0.8408964152537145,
 0.8254605646966103,
 0.8282477531331041,
 0.9764540896763105,
 0.8551943936095625,
 0.8679693563144288,
 0.8857000285382948,
 0.8172805439823428]

In [ ]:
bleu_score = sum(bleu_score_list) / len(bleu_score_list)
print(f"bleu_score: {bleu_score}")

bleu_score: 0.862294521090141


In [ ]:
def chat_with_bot():
    GREETING_INPUTS = ["Chào Bot","hi","hey"]
    GREETING_RESPONSES = ["hi", "hey", "hello", "Tôi rất vui, khi bạn muốn trò chuyện với tôi"]
    def greeting(sentence):
        if sentence.lower() in [g.lower() for g in GREETING_INPUTS]:
            return random.choice(GREETING_RESPONSES)

    flag=True
    print("BOT: Tên tôi là Bot. Tôi sẽ trả lời các câu hỏi của bạn về Chatbots. Nếu bạn muốn thoát, hãy nhập Bye!")
    while(flag==True):
        user_response = input()
        user_response=user_response.lower()
        if(user_response!='bye'):
            if(user_response=='cảm ơn' or user_response=='thanks' ):
                flag=False
                print("BOT: Không có gì? Rất vui được trò chuyện với bạn.")
            else:
                if(greeting(user_response)!=None):
                    print("BOT: "+ greeting(user_response))
                else:
                    response_bot =""
                    if('và' in user_response.split()):
                        user_res= user_response.split(" và ")
                        for i in range(len(user_res)):
                            if i == 0 or i % 2 == 0:
                                response_bot = response_bot + removePunctuations(response(user_res[i]))
                            else:
                                response_bot = response_bot + ", " + removePunctuations(response(user_res[i]))
                        print("BOT: ", end="")
                        print(response_bot)
                    else:
                        print("BOT: ", end="")
                        print(response(user_response))
                    
        else:
            
            flag=False
            print("BOT: Bye!")

In [ ]:
chat_with_bot()

BOT: Tên tôi là Bot. Tôi sẽ trả lời các câu hỏi của bạn về Chatbots. Nếu bạn muốn thoát, hãy nhập Bye!
hi
BOT: Tôi rất vui, khi bạn muốn trò chuyện với tôi
bạn dang làm gì?
BOT: Giang viên
Bạn học chuyên ngành nào?
BOT: Học Mỹ Thuật Công Nghiệp
Bạn quê ở thành phố nào?
BOT: HCM
bye
BOT: Bye!


**Tài Liệu Tham Khảo**
<ul>
    <li>https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html</li>
    <li>https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html</li>
</ul>